In [1]:
%cd /content/drive/My Drive/BIO/dataset

/content/drive/My Drive/BIO/dataset


In [2]:
import glob
import gzip
import os
import tarfile
import time
import warnings
from urllib.request import urlretrieve

import pandas as pd

import keras
from keras.applications import DenseNet121, ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten
from keras.metrics import AUC
from keras.models import load_model, Model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
'''

'''

CLASSES = [
  'Hernia',
  'Pneumonia',
  'Fibrosis',
  'Edema',
  'Emphysema',
  'Cardiomegaly',
  'Pleural_Thickening',
  'Consolidation',
  'Pneumothorax',
  'Mass',
  'Nodule',
  'Atelectasis',
  'Effusion',
  'Infiltration'
]

In [4]:
df = pd.read_csv("Data_Entry_2017.csv")
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [5]:
df['Finding Labels'] = df['Finding Labels'].apply(lambda s: s.split('|'))
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,[No Finding],0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,[Hernia],0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [6]:
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('Finding Labels')),
                          columns=mlb.classes_,
                          index=df.index))
df.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,00000003_000.png,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [7]:
df = df[['Image Index'] + CLASSES]
df.head()

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration
0,00000001_000.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,00000001_001.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,00000001_002.png,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,00000002_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00000003_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# full_dir = "{0}{1}/full".format(ROOT_DIR, DATA_PATH)
# full_dir

'./data/full'

In [9]:
datagen_file = 'labels.csv'
datagen_file

'labels.csv'

In [ ]:
img_paths =  glob.glob('images_*/*/*.png', recursive=True)
df['Image Index'] = df['Image Index'].apply(lambda x: next(p for p in img_paths if x in p))

In [11]:
df

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration
0,00000001_000.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,00000001_001.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,00000001_002.png,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,00000002_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00000003_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,0,1,0,0,0,0,0,0,0,1,0,0,0,0
112116,00030802_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112117,00030803_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112118,00030804_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
df.to_csv(datagen_file, index=False)
df.head()

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration
0,00000001_000.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,00000001_001.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,00000001_002.png,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,00000002_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00000003_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
df2 = df.sample(frac=1.0)
df2

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration
29466,00007670_026.png,0,0,0,0,0,0,0,1,0,0,0,0,1,0
62675,00015530_003.png,0,0,0,0,0,0,0,0,0,0,0,1,1,1
70421,00017362_017.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
79601,00019550_008.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1
46032,00011793_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77991,00019150_009.png,0,0,0,0,0,0,0,0,0,0,0,0,0,1
41296,00010693_023.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61162,00015080_002.png,0,0,0,0,0,0,0,0,0,0,0,1,1,0
83486,00020485_002.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
train_df, test_df = train_test_split(df2, test_size=0.2)

print('Training/Validation Samples:  {}'.format(len(train_df)))
print('Test Samples:  {}'.format(len(test_df)))

Training/Validation Samples:  89696
Test Samples:  22424


In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
ROOT_DIR = '.'
DATA_PATH = '/data'
CHECKPOINT_PATH = '/models'

SAMPLE_RATE = 1.00
EPOCHS = 50
BATCH_SIZE = 32
CHECKPOINT_RATE = 2

In [21]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    subset='training',
    batch_size=32,
    shuffle=True,
    class_mode='raw',
    #classes=CLASSES,
    target_size=(224, 224),
)

Found 0 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 89696 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [19]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    subset='validation',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 0 validated image filenames.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 89696 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)
